In [2]:
!pip install pypsa

In [3]:
import pypsa
import numpy as np
import gc
import pandas as pd
import os
import pickle
from google.colab import drive
import xarray as xr # Import xarray here

# Define a simple, picklable representation for linear expressions
class PicklableLinearExpr:
    def __init__(self, vars, coeffs, const=0):
        self.vars = vars
        self.coeffs = coeffs
        self.const = const

    def __repr__(self):
        terms = [f"{coeff}*var_id_{var}" for var, coeff in zip(self.vars, self.coeffs)]
        expr_str = " + ".join(terms)
        if self.const != 0:
            expr_str += f" + {self.const}"
        return expr_str


def _variable_constraint_mapping(network_file, constraints_to_skip):
    """
    Initialize all optimization components in one pass to avoid creating multiple models.
    This method:
    1. Creates the optimization model once
    2. Extracts objective components (vars, coeffs, const)
    3. Creates the variable ID to name mapping
    4. Extracts constraints
    5. Cleans up the model
    """
    network = pypsa.Network(network_file)
    # Create model once - this is an expensive operation
    temp_model = network.optimize.create_model()

    # Create variable ID to name mapping
    var_id_to_name = {}
    for var_name, variable in temp_model.variables.items():
        # Get the variable labels (IDs) for this variable
        var_labels = variable.labels

        if hasattr(var_labels, 'values'):
            # Multi-dimensional variable
            labels_flat = var_labels.values.flatten()
            coords = variable.labels.coords
            for i, label in enumerate(labels_flat):
                if label != -1:  # -1 means no variable
                    # Create a name that includes the index for multi-dim variables
                    if len(coords) > 0:
                        # Get the coordinate values for this flat index
                        unravel_idx = np.unravel_index(i, var_labels.shape)
                        coord_values = []
                        for dim_idx, dim_name in enumerate(var_labels.dims):
                            coord_val = coords[dim_name].values[unravel_idx[dim_idx]]

                            # Handle datetime64 values properly
                            if isinstance(coord_val, np.datetime64) or hasattr(coord_val, 'strftime'):
                                # Convert datetime to string in ISO format
                                try:
                                    coord_val = pd.Timestamp(coord_val).isoformat()
                                except:
                                    # Fallback if conversion fails
                                    coord_val = str(coord_val)

                            coord_values.append(f"{dim_name}={coord_val}")

                        full_name = f"{var_name}[{','.join(coord_values)}]"
                    else:
                        full_name = f"{var_name}[{i}]"
                    var_id_to_name[label] = full_name
        else:
            # Scalar variable
            var_id_to_name[var_labels] = var_name

    # Store constraint information
    constraints = {}
    for name, constraint_group in temp_model.constraints.items():
        # Corrected condition to skip desired constraints
        if name in constraints_to_skip:
            continue
        # Check if this is a constraint group with multiple individual constraints
        if hasattr(constraint_group.lhs, 'shape') and len(constraint_group.lhs.shape) > 0:
            # This is a constraint group with multiple individual constraints
            # We need to extract each individual constraint

            # Get the dimensions of the constraint group
            dims = constraint_group.lhs.dims if hasattr(constraint_group.lhs, 'dims') else []

            # If it has dimensions, iterate through each individual constraint
            if dims:
                # Get coordinate values for each dimension
                coords = {}
                for dim in dims:
                    if hasattr(constraint_group.lhs, 'coords') and dim in constraint_group.lhs.coords:
                        coords[dim] = constraint_group.lhs.coords[dim].values

                # Create a flat iterator through all combinations of coordinates
                if coords:
                    try:
                        # Create all combinations of coordinate indices - only use dimensions that exist in coords
                        valid_dims = [dim for dim in dims if dim in coords]
                        if not valid_dims:
                            # No valid dimensions found, skip this constraint group
                            print(f"Warning: No valid dimensions found for constraint {name}")
                            continue

                        # Create shape tuple for ndindex
                        shape_tuple = tuple(len(coords[dim]) for dim in valid_dims)
                        if not shape_tuple:
                            # Empty shape tuple, skip this constraint group
                            print(f"Warning: Empty shape tuple for constraint {name}")
                            continue

                        # Create iterator
                        indices = np.ndindex(shape_tuple)

                        # Iterate through all combinations
                        for idx in indices:
                            try:
                                # Create a key for this specific constraint
                                coord_values = []
                                for i, dim in enumerate(valid_dims):
                                    coord_values.append(f"{dim}={coords[dim][idx[i]]}")

                                specific_key = f"{name}[{','.join(coord_values)}]"

                                # Extract the specific constraint values - with error handling
                                try:
                                    # For LHS
                                    if hasattr(constraint_group.lhs.vars, '__getitem__') and hasattr(constraint_group.lhs, 'coeffs'):
                                        # For linear expressions - safely get values
                                        try:
                                            lhs_vars_indexed = constraint_group.lhs.vars[idx]
                                            lhs_coeffs_indexed = constraint_group.lhs.coeffs[idx]

                                            # Check if they are DataArrays and flatten accordingly
                                            lhs_vars = lhs_vars_indexed.values.flatten() if isinstance(lhs_vars_indexed, xr.DataArray) else lhs_vars_indexed.flatten()
                                            lhs_coeffs = lhs_coeffs_indexed.values.flatten() if isinstance(lhs_coeffs_indexed, xr.DataArray) else lhs_coeffs_indexed.flatten()

                                        except Exception as e:
                                            print(f"Warning: Error accessing constraint values for {specific_key}: {e}")
                                            continue

                                        # Create a picklable representation
                                        specific_lhs = PicklableLinearExpr(lhs_vars, lhs_coeffs)

                                        # Add constant if it exists - safely
                                        if hasattr(constraint_group.lhs, 'const'):
                                            try:
                                                if hasattr(constraint_group.lhs.const, '__getitem__'):
                                                    const_val = constraint_group.lhs.const[idx]
                                                else:
                                                    const_val = constraint_group.lhs.const
                                                specific_lhs.const = const_val
                                            except Exception as e:
                                                # If error accessing const, just use 0
                                                specific_lhs.const = 0
                                                print(f"Warning: Error accessing const for {specific_key}: {e}")
                                    else:
                                        # For simple values that are not linear expressions
                                        try:
                                            if hasattr(constraint_group.lhs, '__getitem__'):
                                                specific_lhs = constraint_group.lhs[idx]
                                            else:
                                                specific_lhs = constraint_group.lhs
                                        except Exception as e:
                                            print(f"Warning: Error accessing LHS for {specific_key}: {e}")
                                            continue

                                    # For RHS - safely
                                    try:
                                        if hasattr(constraint_group.rhs, '__getitem__'):
                                            rhs_val = constraint_group.rhs[idx]
                                        else:
                                            rhs_val = constraint_group.rhs
                                        specific_rhs = rhs_val
                                    except Exception as e:
                                        print(f"Warning: Error accessing RHS for {specific_key}: {e}")
                                        continue

                                    # For sign - safely
                                    try:
                                        if hasattr(constraint_group.sign, '__getitem__'):
                                            sign_val = constraint_group.sign[idx].values.item()
                                        else:
                                            sign_val = constraint_group.sign
                                        specific_sign = sign_val
                                    except Exception as e:
                                        print(f"Warning: Error accessing sign for {specific_key}: {e}")
                                        specific_sign = '>=' # Default sign

                                    # Store this specific constraint
                                    constraints[specific_key] = {
                                        'lhs': specific_lhs,
                                        'rhs': specific_rhs,
                                        'sign': specific_sign
                                    }
                                except Exception as e:
                                    print(f"Warning: Error processing constraint {specific_key}: {e}")
                                    continue
                            except Exception as e:
                                print(f"Warning: Error creating key for constraint: {e}")
                                continue
                    except Exception as e:
                        print(f"Warning: Error creating indices for constraint {name}: {e}")
                        continue
            else: #no case handling for when no dimensions but still has shape
                print(f"Warning: No dimensions found for constraint {name}")
                continue
        else:
            # This is a single constraint, store it directly
            try:
                constraints[name] = {
                    'lhs': constraint_group.lhs.copy() if hasattr(constraint_group.lhs, 'copy') else constraint_group.lhs,
                    'rhs': constraint_group.rhs.copy() if hasattr(constraint_group.rhs, 'copy') else constraint_group.rhs,
                    'sign': constraint_group.sign
                }
            except Exception as e:
                print(f"Warning: Error storing single constraint {name}: {e}")
                continue


    # Clean up to free memory
    del temp_model
    gc.collect()

    return var_id_to_name, constraints

def save_mappings(var_id_to_name, constraints, network_file, output_dir="var_constraint_map"):
    """
    Save the variable ID to name mapping and constraints to files in Google Drive.

    Parameters:
    -----------
    var_id_to_name : dict
        Mapping from variable IDs to variable names
    constraints : dict
        Dictionary of constraints
    network_file : str
        Path to the network file used to create the mappings
    output_dir : str, optional
        Directory relative to Google Drive MyDrive to save the mapping files to

    Returns:
    --------
    tuple : (var_map_path, constraints_path) - Paths to the saved mapping files
    """
    # Mount Google Drive
    drive.mount('/content/drive')

    # Create the full path to Google Drive directory
    gdrive_base = '/content/drive/MyDrive/Colab_Notebooks'
    full_output_dir = os.path.join(gdrive_base, output_dir)

    # Create output directory if it doesn't exist
    os.makedirs(full_output_dir, exist_ok=True)

    # Get the network filename without path or extension
    network_name = os.path.basename(network_file)
    network_name = os.path.splitext(network_name)[0]

    # Create output filenames
    var_map_file = os.path.join(full_output_dir, f"{network_name}_var_id_to_name.pkl")
    constraints_file = os.path.join(full_output_dir, f"{network_name}_constraints.pkl")

    # Save mappings to files
    with open(var_map_file, 'wb') as f:
        pickle.dump(var_id_to_name, f)

    with open(constraints_file, 'wb') as f:
        pickle.dump(constraints, f)

    print(f"Saved variable mapping to: {var_map_file}")
    print(f"Saved constraints to: {constraints_file}")

    return var_map_file, constraints_file

def load_mappings(network_file, input_dir="var_constraint_map"):
    """
    Load previously saved variable ID to name mapping and constraints from files in Google Drive.

    Parameters:
    -----------
    network_file : str
        Path to the network file used to create the mappings
    input_dir : str, optional
        Directory relative to Google Drive MyDrive where the mapping files are stored

    Returns:
    --------
    tuple : (var_id_to_name, constraints) - The loaded mappings
    """
    # Mount Google Drive
    drive.mount('/content/drive')

    # Create the full path to Google Drive directory
    gdrive_base = '/content/drive/MyDrive/Colab_Notebooks'
    full_input_dir = os.path.join(gdrive_base, input_dir)

    # Get the network filename without path or extension
    network_name = os.path.basename(network_file)
    network_name = os.path.splitext(network_name)[0]

    # Create input filenames
    var_map_file = os.path.join(full_input_dir, f"{network_name}_var_id_to_name.pkl")
    constraints_file = os.path.join(full_input_dir, f"{network_name}_constraints.pkl")

    # Check if files exist
    if not os.path.exists(var_map_file) or not os.path.exists(constraints_file):
        raise FileNotFoundError(f"Mapping files for {network_name} not found in {full_input_dir}")

    # Load mappings from files
    with open(var_map_file, 'rb') as f:
        var_id_to_name = pickle.load(f)

    with open(constraints_file, 'rb') as f:
        constraints = pickle.load(f)

    print(f"Loaded variable mapping from: {var_map_file}")
    print(f"Loaded constraints from: {constraints_file}")

    return var_id_to_name, constraints

In [4]:
def main():
    # Example usage
    drive.mount('/content/drive')
    network_file = "/content/drive/MyDrive/Colab_Notebooks/networks_1_week/elec_s_5_ec_lc1.0_3h.nc"
    constraints_to_skip = [
        "StorageUnit-fix-p_dispatch-lower",
        "StorageUnit-fix-p_dispatch-upper",
        "StorageUnit-fix-p_store-lower",
        "StorageUnit-fix-p_store-upper",
        "StorageUnit-fix-state_of_charge-lower",
        "StorageUnit-fix-state_of_charge-upper",
        "StorageUnit-energy_balance"
    ]

    # Create mappings
    var_id_to_name, constraints = _variable_constraint_mapping(network_file, constraints_to_skip)

    # Save mappings
    var_map_file, constraints_file = save_mappings(var_id_to_name, constraints, network_file)

    print(f"Created variable ID to name mapping with {len(var_id_to_name)} entries")
    print(f"Created constraints mapping with {len(constraints)} entries")
    print(f"Saved variable mapping to {var_map_file}")
    print(f"Saved constraints mapping to {constraints_file}")

In [5]:
main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA1 0', 'ZA2 0', 'ZA3 0'], dtype='object', name='Bus')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved variable mapping to: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_5_ec_lc1.0_3h_var_id_to_name.pkl
Saved constraints to: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_5_ec_lc1.0_3h_constraints.pkl
Created variable ID to name mapping with 3416 entries
Created constraints mapping with 5880 entries
Saved variable mapping to /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_5_ec_lc1.0_3h_var_id_to_name.pkl
Saved constraints mapping to /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_5_ec_lc1.0_3h_constraints.pkl


In [13]:
%debug

> /usr/local/lib/python3.11/dist-packages/pandas/core/arrays/datetimes.py(547)_box_func()
    545     def _box_func(self, x: np.datetime64) -> Timestamp | NaTType:
    546         # GH#42228
--> 547         value = x.view("i8")
    548         ts = Timestamp._from_value_and_reso(value, reso=self._creso, tz=self.tz)
    549         return ts

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
